In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: auto; }</style>"))

In [ ]:
import my.emfit as emfit

In [ ]:
df = emfit.dataframe()

In [ ]:
df

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

In [ ]:
from bokeh.models import ColumnDataSource


# df = df[df['error'].isnull()]
# df = df.set_index('date')

# print(df[:10])

# raise RuntimeError
# todo make index??
# mean = cs.rolling(f'{d}D').mean()

# TODO ugh. figure out how pandas determines the index type..
# import pandas as pd
# df.index = pd.to_datetime(df.index, utc=True)


# see https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#basic-tooltips
# TODO date is displayed as unix time
from bokeh.models import HoverTool
hover = HoverTool(
    tooltips=[
        ( 'date',   '@date{%F}'            ),
    ],

    formatters={
        '@date'        : 'datetime', # use 'datetime' formatter for '@date' field
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)

# TODO reuse functions for adding moving averages? Perhaps they should operate on dataframes to be framework independent?
p = figure(x_axis_type='datetime', plot_width=2000)
p.add_tools(hover)


# TODO https://github.com/bokeh/bokeh/blob/master/examples/app/sliders.py

# TODO if there are Nans, it fails to plot anything??

m7d  = df[['avg_hr']].rolling('7D' ).mean()
m30d = df[['avg_hr']].rolling('30D').mean()

p.scatter(x='date', y='avg_hr', source=ColumnDataSource(data=df  ), color='blue' , legend_label='HR'              )
p.line   (x='date', y='avg_hr', source=ColumnDataSource(data=m7d ), color='red'  , legend_label='HR (7  days avg)')
p.line   (x='date', y='avg_hr', source=ColumnDataSource(data=m30d), color='green', legend_label='HR (30 days avg)')


p.legend.click_policy = 'hide'


show(p)